In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

In [2]:
# !pip install wandb -qqq
import os
os.environ["WANDB_API_KEY"] = "0f780ac8a470afe6cb7fc474ff3794772c660465"

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"

import wandb
# wandb.login()
api = wandb.Api()





# selected_tag = 'Pred'

runs = api.runs("hokarami/TEEDAM_supervised")


In [3]:
import pandas as pd 


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):




    summary_list, config_list, name_list, path_list = [], [], [], []
    for run in all_runs: 

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
        path_list.append(run.path)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list,
        "path": path_list

        })

    # runs_df.to_csv("project.csv")
    return runs_df

In [4]:
# q = runs_df['name'].str.contains('\[R2-')
# runs_df = runs_df[q]


runs_df = dl_runs(runs, selected_tag='RD74-table')

runs_df
runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


,summary,config,name,path
0,{'Valid-NextType(ML)/auc-ovo-weighted': 0.8034...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__nextmark-concat]2652237,"[hokarami, TEEDAM_supervised, 7d21r69p]"
1,{'Valid-NextType(ML)/f1-weighted': 0.262404408...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__pp_single_mark-concat]2642838,"[hokarami, TEEDAM_supervised, s1j6o0zy]"
2,"{'Test-CIF/NLL': 64609.86328125, 'Valid-pred_l...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__pp_ml-concat]2648578,"[hokarami, TEEDAM_supervised, 15hmxzvm]"
3,{'Test-NextTime/rmse_norm': 13.142569186748654...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__none-concat]2648879,"[hokarami, TEEDAM_supervised, hsdecn83]"
4,"{'num_params': 177074, 'Valid-pred_label/AUROC...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-DA__base-concat]2647742,"[hokarami, TEEDAM_supervised, 3rsdp74k]"
5,"{'LR': 1e-05, 'Valid-pred_label/loss': 0, 'tim...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__nextmark-concat]2641256,"[hokarami, TEEDAM_supervised, rzhi6bsr]"
6,"{'Test-CIF/NLL': 32814.78515625, 'Best-Test-pr...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__pp_single_mark-concat]2644670,"[hokarami, TEEDAM_supervised, miivn0oo]"
7,"{'Obj': 0.5009140767824497, 'Test-CIF/NLL': 66...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__pp_ml-concat]2645914,"[hokarami, TEEDAM_supervised, 1xxbhn0y]"
8,"{'Valid-pred_label/AUPRC': 0.5058257026821624,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-TEDA__none-concat]2642178,"[hokarami, TEEDAM_supervised, vxplzu9e]"
9,"{'Best-Valid-NextType(ML)/recall-weighted': 0,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-DA__base-concat]2649487,"[hokarami, TEEDAM_supervised, 25nz6cxm]"


dict_keys(['Valid-NextType(ML)/auc-ovo-weighted', 'Best-Valid-pred_label/precision-binary', 'LR', 'Valid-pred_label/AUROC', 'Test-pred_label/f1-binary', 'Best-Test-pred_label/AUPRC', 'Best-Test-pred_label/f1-binary', 'Best-Valid-NextType(ML)/f1-weighted', '_timestamp', 'max_obj_val', 'Precision-Recall Curve', 'Best-Valid-pred_label/loss', 'Test-NextType(ML)/auc-PR-weighted', 'Test-NextType(ML)/f1-weighted', 'Test-pred_label/recall-binary', '_wandb', 'num_params', 'Valid-NextTime/RMSE', 'Best-Valid-pred_label/AUPRC', 'Best-Valid-pred_label/AUROC', 'Best-Valid-NextTime/rmse_norm', 'Best-Test-NextType(ML)/recall-weighted', 'Best-Valid-NextType(ML)/precision-weighted', 'best_epoch', 'Valid-NextTime/rmse_norm', 'Best-Valid-pred_label/recall-binary', 'Test-NextType(ML)/precision-weighted', '_step', 'Valid-NextTime/msae', 'Best-Test-pred_label/ACC', 'Best-Valid-NextTime/msae', 'Best-Valid-pred_label/f1-binary', 'Best-Test-NextType(ML)/auc-ovo-weighted', 'Obj', 'Best-Test-NextTime/rmse_norm', 

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'INPUT', 'epoch', 'noise', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'OUTPUT', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'tl_tag', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'log_freq', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'diag_offset', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'NOISE_config', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'wandb_project', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'dict_map_events', 'dict_map_states', 'next_time_co

# Table I
Effect of time concatenation and single+mark

In [5]:
runs = api.runs("hokarami/TEEDAM_unsupervised_timeCat")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt


,summary,config,name,path
0,"{'Valid-NextType(MC)/acc': 0.5067098105126473,...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[H70-TE__pp_mc-sum-d1]1888307-,"[hokarami, TEEDAM_unsupervised_timeCat, swclgn94]"
1,"{'Test-NextType(ML)/f1-weighted-CIF': 0, 'Vali...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_ml-concat-d1]1846111-,"[hokarami, TEEDAM_unsupervised_timeCat, ngompq5p]"
2,{'Test-NextType(ML)/f1-weighted': 0.1698214655...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_single_mark-sum-d1]1847764-,"[hokarami, TEEDAM_unsupervised_timeCat, usls7sls]"
3,"{'Test-NextTime/RMSE': 3.4010276304092084, 'ti...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_single_mark-concat-d1]1848546-,"[hokarami, TEEDAM_unsupervised_timeCat, rq60tih1]"
4,{'Best-Test-CIF/LL-#events': -4.82887770046604...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_ml-sum-d1]1843295-,"[hokarami, TEEDAM_unsupervised_timeCat, mtuvvfln]"
...,...,...,...,...
374,{'Valid-NextType(ML)/precision-weighted': 0.34...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__nextmark-concat-d-2]1785157-,"[hokarami, TEEDAM_unsupervised_timeCat, j9z1o2jy]"
375,{'Test-NextType(MC)/f1-macro': 0.4492456530605...,"{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[H70-TE__nextmark-concat-d-2]1787979-,"[hokarami, TEEDAM_unsupervised_timeCat, imm1g4my]"
376,{'Valid-NextType(ML)/precision-weighted': 0.28...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__nextmark-concat-d-2]1787649-,"[hokarami, TEEDAM_unsupervised_timeCat, me3gzd7s]"
377,{'Test-NextTime/rmse_norm': 293.05339260175583...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__nextmark-concat-d-2]1786183-,"[hokarami, TEEDAM_unsupervised_timeCat, i0kftsh7]"


In [45]:
i_groupby_row =['dataset']
i_groupby_col = ['time_enc']
i_groupby_col=['mod','diag_offset','time_enc']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Best-Test-NextType(ML)/auc-ovo-weighted','Best-Test-NextType(MC)/f1-weighted']
dict_rename_metrics = {'Best-Test-CIF/LL-#events':'LL/#events', 'Best-Test-NextType(ML)/auc-ovo-weighted':'AUROC','Best-Test-NextType(MC)/f1-weighted':'F1-Score'}
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)

# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

df = df.rename(columns=dict_rename_metrics)
i_metric = list(dict_rename_metrics.values())

# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')


a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'  ).reset_index() #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()

a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'



df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')



# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

df_result = df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# remove irrelevant metrics
df_result = df_result.replace('nan (nan)', np.nan).dropna(axis=0,how='all')
df_result


value                                         \
mod                        PP(ML/MC)                                          
diag_offset                       -2                        -1                
time_enc                      concat          sum       concat          sum   
dataset      Metric                                                           
Retweets(MC) AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score      0.53(nan)    0.35(nan)    0.52(nan)    0.35(nan)   
             LL/#events  -10.28(nan)  -22.16(nan)  -12.02(nan)  -18.85(nan)   
Retweets(ML) AUROC        0.74(0.01)   0.72(0.01)   0.74(0.01)   0.72(0.01)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -1.34(0.03)  -1.58(0.03)  -1.50(0.05)  -1.58(0.03)   
SO           AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score     0.33(0.01)   0.33(0.01)   0.33(0.01)   0.32(0.00)   
             LL/#events  -2.02(0.02)  -2.02(0.02)  -2.02(0.01)  -2.04(0.01)   
Synthea      AUROC        0.58(0.02)   0.59(0.05)   0.60(0.02)   0.61(0.04)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -5.49(0.49)  -5.43(0.48)  -5.54(0.46)  -5.35(0.41)   

                                                                             \
mod                                                                           
diag_offset                        0                         1                
time_enc                      concat          sum       concat          sum   
dataset      Metric                                                           
Retweets(MC) AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score      0.53(nan)    0.45(nan)    0.44(nan)    0.44(nan)   
             LL/#events  -16.29(nan)  -13.22(nan)  -12.03(nan)  -21.68(nan)   
Retweets(ML) AUROC        0.74(0.02)   0.71(0.02)   0.66(0.03)   0.66(0.03)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -1.49(0.03)  -1.60(0.04)  -1.67(0.04)  -1.68(0.05)   
SO           AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score     0.32(0.00)   0.32(0.00)   0.32(0.01)   0.32(0.00)   
             LL/#events  -2.04(0.01)  -2.05(0.01)  -2.08(0.05)  -2.07(0.02)   
Synthea      AUROC        0.60(0.05)   0.60(0.05)   0.57(0.06)   0.61(0.05)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -5.50(0.46)  -5.33(0.42)  -5.61(0.41)  -5.26(0.39)   

                                                 ...                          \
mod                              AE              ...                           
diag_offset                      -2              ...           1               
time_enc                     concat         sum  ...      concat         sum   
dataset      Metric                              ...                           
Retweets(MC) AUROC         nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             F1-Score     0.63(nan)   0.62(nan)  ...   0.59(nan)   0.59(nan)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
Retweets(ML) AUROC       0.71(0.02)  0.69(0.03)  ...  0.66(0.03)  0.66(0.03)   
             F1-Score      nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
SO           AUROC         nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             F1-Score    0.38(0.01)  0.36(0.01)  ...  0.32(0.01)  0.33(0.00)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
Synthea      AUROC       0.90(0.00)  0.89(0.01)  ...  0.84(0.05)  0.87(0.03)   
             F1-Score      nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan

In [59]:
df_result2 = df_result.transpose().sort_values(by=i_groupby_col,axis=0)

# df_result2

df_result2 = df_result2.replace('nan(nan)', np.nan).replace(r'\(nan\)$', '',regex=True).dropna(how='all', axis=1).replace(np.nan, '')

df_result2

dataset                               Retweets(MC)            Retweets(ML)  \
Metric                                    F1-Score LL/#events        AUROC   
      mod        diag_offset time_enc                                        
value AE         -2          concat           0.63              0.71(0.02)   
                             sum              0.62              0.69(0.03)   
                 -1          concat           0.63              0.71(0.02)   
                             sum              0.62              0.69(0.02)   
                  0          concat           0.63              0.70(0.03)   
                             sum              0.61              0.69(0.03)   
                  1          concat           0.59              0.66(0.03)   
                             sum              0.59              0.66(0.03)   
      PP(ML/MC)  -2          concat           0.53     -10.28   0.74(0.01)   
                             sum              0.35     -22.16   0.72(0.01)   
                 -1          concat           0.52     -12.02   0.74(0.01)   
                             sum              0.35     -18.85   0.72(0.01)   
                  0          concat           0.53     -16.29   0.74(0.02)   
                             sum              0.45     -13.22   0.71(0.02)   
                  1          concat           0.44     -12.03   0.66(0.03)   
                             sum              0.44     -21.68   0.66(0.03)   
      PP(single) -2          concat           0.57      -6.79   0.71(0.02)   
                             sum              0.56      -7.96   0.68(0.04)   
                 -1          concat           0.53      -6.64   0.70(0.03)   
                             sum              0.54     -10.34   0.68(0.03)   
                  0          concat           0.57      -6.58   0.70(0.03)   
                             sum              0.54     -10.03   0.68(0.03)   
                  1          concat           0.56      -6.44   0.66(0.03)   
                             sum              0.32      -9.02   0.66(0.03)   

dataset                                                     SO               \
Metric                                  LL/#events    F1-Score   LL/#events   
      mod        diag_offset time_enc                                         
value AE         -2          concat                 0.38(0.01)                
                             sum                    0.36(0.01)                
                 -1          concat                 0.39(0.01)                
                             sum                    0.35(0.02)                
                  0          concat                 0.38(0.02)                
                             sum                    0.34(0.01)                
                  1          concat                 0.32(0.01)                
                             sum                    0.33(0.00)                
      PP(ML/MC)  -2          concat    -1.34(0.03)  0.33(0.01)  -2.02(0.02)   
                             sum       -1.58(0.03)  0.33(0.01)  -2.02(0.02)   
                 -1          concat    -1.50(0.05)  0.33(0.01)  -2.02(0.01)   
                             sum       -1.58(0.03)  0.32(0.00)  -2.04(0.01)   
                  0          concat    -1.49(0.03)  0.32(0.00)  -2.04(0.01)   
                             sum       -1.60(0.04)  0.32(0.00)  -2.05(0.01)   
                  1          concat    -1.67(0.04)  0.32(0.01)  -2.08(0.05)   
                             sum       -1.68(0.05)  0.32(0.00)  -2.07(0.02)   
      PP(single) -2          concat     1.99(0.06)  0.37(0.01)  -0.51(0.01)   
                             sum        1.86(0.09)  0.35(0.00)  -0.54(0.01)   
                 -1          concat     2.02(0.05)  0.38(0.01)  -0.50(0.01)   
                             sum        1.87(0.10)  0.35(0.01)  -0.54(0.01)   
                  0          concat     2.08(0.05)  0.36(0.01)  -0.53(0.01) 

In [28]:
a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate('mean').round(3).reset_index()


a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'

a1

q = a1['mod']=='PP(ML/MC)'

a1.loc[q,'LL/#events'].corr(a1.loc[q,'LL/#events'])

,dataset,mod,diag_offset,time_enc,LL/#events,AUROC,F1-Score
0,Retweets(MC),PP(ML/MC),-2,concat,-10.283,NaN,0.532
1,Retweets(MC),PP(ML/MC),-2,sum,-22.161,NaN,0.353
2,Retweets(MC),PP(ML/MC),-1,concat,-12.020,NaN,0.522
3,Retweets(MC),PP(ML/MC),-1,sum,-18.847,NaN,0.354
4,Retweets(MC),PP(ML/MC),0,concat,-16.285,NaN,0.527
...,...,...,...,...,...,...,...
91,Synthea,PP(single),-1,sum,-2.064,0.643,NaN
92,Synthea,PP(single),0,concat,-1.971,0.643,NaN
93,Synthea,PP(single),0,sum,-2.104,0.637,NaN
94,Synthea,PP(single),1,concat,-1.970,0.617,NaN


1.0

# Table II

In [33]:
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt

,summary,config,name,path
0,{'Best-Valid-pred_label/precision-binary': 0.1...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70H-TEnoise__pp_single_mark-concat-d0]2085332,"[hokarami, TEEDAM_unsupervised, jucp58jt]"
1,"{'Valid-pred_label/ACC': 0.9143145161290324, '...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70H-TEnoise__pp_single_mark-concat-d0]2089506,"[hokarami, TEEDAM_unsupervised, o4wrmhhj]"
2,{'Test-pred_label/precision-binary': 0.1005643...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70H-TEDA__pp_single_mark-concat-d0]2083522,"[hokarami, TEEDAM_unsupervised, gctg32i9]"
3,"{'_wandb': {'runtime': 5138}, 'Best-Valid-Next...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70H-TEDA__pp_single_mark-concat-d0]2089954,"[hokarami, TEEDAM_unsupervised, p15l4sk9]"
4,{'Best-Valid-CIF/LL-#events': -0.6655285938517...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70H-TEnoise__pp_single_mark-concat-d0]2085804,"[hokarami, TEEDAM_unsupervised, mre7wsjj]"
...,...,...,...,...
153,"{'_step': 100, 'Test-CIF/LL-#events': -1.32713...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d0]1993968,"[hokarami, TEEDAM_unsupervised, 9n5cs2dt]"
154,{'Best-Valid-NextType(ML)/auc-ovo-weighted': 0...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d0]1999765,"[hokarami, TEEDAM_unsupervised, kbstsdnm]"
155,{'Best-Valid-NextType(ML)/f1-weighted': 0.3913...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEDA__pp_single_mark-concat-d0]1993078,"[hokarami, TEEDAM_unsupervised, f3kcjkiq]"
156,"{'Test-NextTime/RMSE': 2.327216482334012, 'tim...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d0]1997768,"[hokarami, TEEDAM_unsupervised, yycj47wo]"


In [34]:
i_groupby_row =[]
i_groupby_col = ['time_enc']
i_groupby_col=['dataset','setting','INPUT','diag_offset']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Best-Test-NextType(ML)/auc-ovo-weighted']
dict_rename_metrics = {'Best-Test-CIF/LL-#events':'LL/#events', 'Best-Test-NextType(ML)/auc-ovo-weighted':'AUROC'}
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)

# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

df = df.rename(columns=dict_rename_metrics)
i_metric = list(dict_rename_metrics.values())

# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'  ).reset_index() #  


# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()

# a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
# a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
# a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'





df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')



# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

df_result = df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# remove irrelevant metrics
df_result = df_result.replace('nan (nan)', np.nan).dropna(axis=0,how='all')
df_result


value                                                      \
dataset              P12                                                       
setting              mc1                                    mc2                
INPUT                 TE        TEDAM      TENOISE           TE        TEDAM   
diag_offset            0            0            0            0            0   
Metric                                                                         
AUROC         0.80(0.04)   0.84(0.02)   0.80(0.06)   0.78(0.02)   0.83(0.01)   
LL/#events   -1.34(0.04)  -1.30(0.03)  -1.35(0.05)  -1.35(0.01)  -1.32(0.02)   

                                                                              \
dataset                                                                  P19   
setting                            sc                                    mc1   
INPUT            TENOISE           TE        TEDAM      TENOISE           TE   
diag_offset            0            0            0            0            0   
Metric                                                                         
AUROC         0.82(0.01)   0.73(0.05)   0.77(0.05)   0.73(0.06)   0.78(0.05)   
LL/#events   -1.34(0.01)  -1.41(0.03)  -1.38(0.04)  -1.40(0.04)  -0.87(0.26)   

                                                                              \
dataset                                                                        
setting                                        mc2                             
INPUT              TEDAM      TENOISE           TE        TEDAM      TENOISE   
diag_offset            0            0            0            0            0   
Metric                                                                         
AUROC         0.86(0.02)   0.76(0.05)   0.61(0.05)   0.74(0.11)   0.66(0.05)   
LL/#events   -0.72(0.27)  -0.88(0.24)  -1.17(0.19)  -1.02(0.17)  -1.23(0.26)   

                                                    
dataset                                             
setting               sc                            
INPUT                 TE        TEDAM      TENOISE  
diag_offset            0            0            0  
Metric                                              
AUROC         0.78(0.04)   0.88(0.03)   0.78(0.03)  
LL/#events   -0.94(0.29)  -0.80(0.28)  -0.91(0.27)

In [35]:
df_result.transpose().sort_values(by=i_groupby_col,axis=0)



Metric                                          AUROC   LL/#events
      dataset setting INPUT   diag_offset                         
value P12     mc1     TE      0            0.80(0.04)  -1.34(0.04)
                      TEDAM   0            0.84(0.02)  -1.30(0.03)
                      TENOISE 0            0.80(0.06)  -1.35(0.05)
              mc2     TE      0            0.78(0.02)  -1.35(0.01)
                      TEDAM   0            0.83(0.01)  -1.32(0.02)
                      TENOISE 0            0.82(0.01)  -1.34(0.01)
              sc      TE      0            0.73(0.05)  -1.41(0.03)
                      TEDAM   0            0.77(0.05)  -1.38(0.04)
                      TENOISE 0            0.73(0.06)  -1.40(0.04)
      P19     mc1     TE      0            0.78(0.05)  -0.87(0.26)
                      TEDAM   0            0.86(0.02)  -0.72(0.27)
                      TENOISE 0            0.76(0.05)  -0.88(0.24)
              mc2     TE      0            0.61(0.05)  -1.17(0.19)
                      TEDAM   0            0.74(0.11)  -1.02(0.17)
                      TENOISE 0            0.66(0.05)  -1.23(0.26)
              sc      TE      0            0.78(0.04)  -0.94(0.29)
                      TEDAM   0            0.88(0.03)  -0.80(0.28)
                      TENOISE 0            0.78(0.03)  -0.91(0.27)

In [48]:
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )
df_path = df_filt.path.apply(lambda x:'/'.join(x))

df = pd.concat([df_config, df_summary, df_path],axis=1)

q=(df['dataset']=='P12') & (df['setting']=='mc2') & (df['INPUT']=='TEDAM') & (df['diag_offset']==1) & (df['test_center']=='1')
# df[q]
# list(df.columns)
df.loc[q,'path']

115    hokarami/TEEDAM_unsupervised/zo2u72u2
Name: path, dtype: object

# Table III

In [5]:
runs = api.runs("hokarami/TEEDAM_supervised")
# df_filt = dl_runs(runs, selected_tag='H70HHGGG') # H70PretrainedOnly
df_filt = dl_runs(runs, selected_tag='RD74-table') # H70PretrainedOnly
len(df_filt)


55

In [7]:
i_groupby_row =['dataset','test_center','setting']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['INPUT','OUTPUT']
i_agg = ['split']

# 'knn-ps-mean',
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'   ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col).replace(r'\(nan\)$', '',regex=True)



# a1.to_excel("aaa.xlsx")

,dataset,test_center,setting,INPUT,OUTPUT,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,P19,1,sc,TEDAM,none-mark-label,4,0.509960,0.585141,0.936881
1,P19,1,sc,DAM,none-mark-label,4,0.545994,0.614948,0.928929
2,P19,1,sc,TEDAM,none-mark-label,3,0.578616,0.652080,0.931283
3,P19,1,sc,DAM,none-mark-label,3,0.545181,0.604926,0.917107
4,P19,1,sc,TEDAM,none-mark-label,2,0.503106,0.676728,0.932575
...,...,...,...,...,...,...,...,...,...
117,P12,0,mc2,DAM,none-mark-label,,0.502216,0.484488,0.832434
118,P19,0,mc2,DAM,none-mark-label,,0.274078,0.633099,0.862014
119,P19,0,mc2,TEDAM,none-mark-label,,0.334844,0.467754,0.748770
120,P19,1,mc2,TEDAM,none-mark-label,,0.589313,0.478888,0.896525


Best-Test-pred_label/f1-binary  \
dataset test_center setting INPUT OUTPUT                                           
P12     0           mc1     DAM   none-mark-label                     0.52(0.04)   
                            TEDAM none-mark-label                     0.51(0.04)   
                    mc2     DAM   none-mark-label                      0.50(nan)   
                            TEDAM none-mark-label                       nan(nan)   
                    sc      DAM   none-mark-label                     0.41(0.06)   
                            TEDAM none-mark-label                     0.42(0.03)   
        1           mc1     DAM   none-mark-label                     0.55(0.04)   
                            TEDAM none-mark-label                     0.55(0.04)   
                    mc2     DAM   none-mark-label                      0.54(nan)   
                            TEDAM none-mark-label                      0.54(nan)   
                    sc      DAM   none-mark-label                     0.48(0.05)   
                            TEDAM none-mark-label                     0.42(0.04)   
        2           mc1     DAM   none-mark-label                     0.54(0.04)   
                            TEDAM none-mark-label                     0.54(0.05)   
                    mc2     DAM   none-mark-label                      0.50(nan)   
                            TEDAM none-mark-label                      0.52(nan)   
                    sc      DAM   none-mark-label                     0.46(0.06)   
                            TEDAM none-mark-label                     0.42(0.05)   
P19     0           mc1     DAM   none-mark-label                     0.58(0.03)   
                            TEDAM none-mark-label                     0.55(0.03)   
                    mc2     DAM   none-mark-label                      0.27(nan)   
                            TEDAM none-mark-label                      0.33(nan)   
                    sc      DAM   none-mark-label                     0.62(0.01)   
                            TEDAM none-mark-label                     0.62(0.01)   
        1           mc1     DAM   none-mark-label                     0.57(0.05)   
                            TEDAM none-mark-label                     0.58(0.04)   
                    mc2     DAM   none-mark-label                      0.59(nan)   
                            TEDAM none-mark-label                      0.59(nan)   
                    sc      DAM   none-mark-label                     0.54(0.01)   
                            TEDAM none-mark-label                     0.53(0.03)   

                                                  Best-Test-pred_label/AUPRC  \
dataset test_center setting INPUT OUTPUT                                       
P12     0           mc1     DAM   none-mark-label                 0.51(0.05)   
                            TEDAM none-mark-label                 0.49(0.07)   
                    mc2     DAM   none-mark-label                  0.48(nan)   
                            TEDAM none-mark-label                   nan(nan)   
                    sc      DAM   none-mark-label                 0.34(0.06)   
                            TEDAM none-mark-label                 0.35(0.04)   
        1           mc1     DAM   none-mark-label                 0.54(0.04)   
                            TEDAM none-mark-label                 0.52(0.04)   
                    mc2     DAM   none-mark-label                  0.52(nan)   
                            TEDAM none-mark-label                  0.52(nan)   
                    sc      DAM   none-mark-label                 0.41(0.06)   
                            TEDAM none-mark-label                 0.33(0.04)   
        2           mc1     DAM   none-mark-label                 0.54(0.04)   
                            TEDAM none-mark-label                 0.53(0.06)   
                    mc2     DAM   none-mark-label                  0.46(nan)   
 

Best-Test-pred_label/f1-binary                  \
INPUT                                                  DAM           TEDAM   
OUTPUT                                     none-mark-label none-mark-label   
dataset test_center setting                                                  
P12     0           mc1                         0.52(0.04)      0.51(0.04)   
                    mc2                               0.50             nan   
                    sc                          0.41(0.06)      0.42(0.03)   
        1           mc1                         0.55(0.04)      0.55(0.04)   
                    mc2                               0.54            0.54   
                    sc                          0.48(0.05)      0.42(0.04)   
        2           mc1                         0.54(0.04)      0.54(0.05)   
                    mc2                               0.50            0.52   
                    sc                          0.46(0.06)      0.42(0.05)   
P19     0           mc1                         0.58(0.03)      0.55(0.03)   
                    mc2                               0.27            0.33   
                    sc                          0.62(0.01)      0.62(0.01)   
        1           mc1                         0.57(0.05)      0.58(0.04)   
                    mc2                               0.59            0.59   
                    sc                          0.54(0.01)      0.53(0.03)   

                            Best-Test-pred_label/AUPRC                  \
INPUT                                              DAM           TEDAM   
OUTPUT                                 none-mark-label none-mark-label   
dataset test_center setting                                              
P12     0           mc1                     0.51(0.05)      0.49(0.07)   
                    mc2                           0.48             nan   
                    sc                      0.34(0.06)      0.35(0.04)   
        1           mc1                     0.54(0.04)      0.52(0.04)   
                    mc2                           0.52            0.52   
                    sc                      0.41(0.06)      0.33(0.04)   
        2           mc1                     0.54(0.04)      0.53(0.06)   
                    mc2                           0.46            0.52   
                    sc                      0.43(0.05)      0.34(0.05)   
P19     0           mc1                     0.74(0.04)      0.69(0.08)   
                    mc2                           0.63            0.47   
                    sc                      0.71(0.01)      0.72(0.02)   
        1           mc1                     0.59(0.08)      0.60(0.06)   
                    mc2                           0.47            0.48   
                    sc                      0.60(0.04)      0.62(0.05)   

                            Best-Test-pred_label/AUROC                  
INPUT                                              DAM           TEDAM  
OUTPUT                                 none-mark-label none-mark-label  
dataset test_center setting                                             
P12     0           mc1                     0.84(0.03)      0.83(0.03)  
                    mc2                           0.83             nan  
                    sc                      0.76(0.04)      0.77(0.03)  
        1           mc1                     0.85(0.01)      0.86(0.02)  
                    mc2                           0.85            0.85  
                    sc                      0.80(0.02)      0.77(0.02)  
        2           mc1                     0.85(0.02)      0.85(0.03)  
                    mc2                           0.82            0.85  
                    sc                      0.80(0.04)      0.75(0.03)  
P19     0           mc1                     0.91(0.02)      0.88(0.05)  
                    mc2                           0.86            0.75  
                    sc                     

: 

## T3 new

In [16]:
df_config.columns

df_summary.columns

Index(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof',
       'INPUT', 'epoch', 'noise', 'split', 'state', 'w_pos', 'wandb', 'ES_pat',
       'OUTPUT', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v',
       'tl_tag', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class',
       'w_event', 'log_freq', 'run_name', 'run_path', 'te_d_rnn', 'time_enc',
       'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks',
       'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head',
       'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label',
       'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout',
       'testloader', 'demo_config', 'diag_offset', 'mark_detach',
       'te_n_layers', 'test_center', 'trainloader', 'user_prefix',
       'validloader', 'w_pos_label', 'NOISE_config', 'label_config',
       'sample_label', 'weight_decay', 'hparams2write', 'wandb_project',
       'balanced_batch', 'label_loss_fun', 

Index(['Valid-NextType(ML)/auc-ovo-weighted',
       'Best-Valid-pred_label/precision-binary', 'LR',
       'Valid-pred_label/AUROC', 'Test-pred_label/f1-binary',
       'Best-Test-pred_label/AUPRC', 'Best-Test-pred_label/f1-binary',
       'Best-Valid-NextType(ML)/f1-weighted', '_timestamp', 'max_obj_val',
       'Precision-Recall Curve', 'Best-Valid-pred_label/loss',
       'Test-NextType(ML)/auc-PR-weighted', 'Test-NextType(ML)/f1-weighted',
       'Test-pred_label/recall-binary', '_wandb', 'num_params',
       'Valid-NextTime/RMSE', 'Best-Valid-pred_label/AUPRC',
       'Best-Valid-pred_label/AUROC', 'Best-Valid-NextTime/rmse_norm',
       'Best-Test-NextType(ML)/recall-weighted',
       'Best-Valid-NextType(ML)/precision-weighted', 'best_epoch',
       'Valid-NextTime/rmse_norm', 'Best-Valid-pred_label/recall-binary',
       'Test-NextType(ML)/precision-weighted', '_step', 'Valid-NextTime/msae',
       'Best-Test-pred_label/ACC', 'Best-Valid-NextTime/msae',
       'Best-Valid-pred

In [28]:
i_groupby_row =['dataset','split','run_name']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['INPUT','OUTPUT']
i_agg = ['split']

i_groupby_col=[]
i_agg = []

# 'knn-ps-mean',
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC',
            'Best-Valid-pred_label/f1-binary','Best-Valid-pred_label/AUPRC','Best-Valid-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

# remove the last 6 characters from the run_name
df['run_name'] = df['run_name'].apply(lambda x:x[:-7])


df.sort_values(by=['dataset','split'] + ['Best-Test-pred_label/AUPRC'], ascending=False)
df.sort_values(by=['dataset','run_name']).groupby(['dataset','run_name'])[i_metric].mean()


term

a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'   ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col).replace(r'\(nan\)$', '',regex=True)



# a1.to_excel("aaa.xlsx")

,dataset,split,run_name,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC,Best-Valid-pred_label/f1-binary,Best-Valid-pred_label/AUPRC,Best-Valid-pred_label/AUROC
32,P19,4,[RD74-TEDA__pp_ml-concat],0.470588,0.474048,0.860270,0.498567,0.506206,0.880937
30,P19,4,[RD74-TEDA__nextmark-concat],0.485294,0.472690,0.851787,0.509934,0.497564,0.874577
29,P19,4,[RD74-TL-TEDA__pp_single_mark-concat],0.494297,0.470018,0.855518,0.511945,0.492756,0.881374
31,P19,4,[RD74-TEDA__pp_single_mark-concat],0.498024,0.469760,0.847487,0.529210,0.502296,0.876258
33,P19,4,[RD74-TEDA__none-concat],0.479452,0.467694,0.861315,0.480480,0.503415,0.879524
34,P19,4,[RD74-DA__base-concat],0.464646,0.466094,0.846376,0.530864,0.506672,0.879983
37,P19,3,[RD74-TEDA__pp_ml-concat],0.455621,0.478926,0.881481,0.495468,0.499621,0.862470
35,P19,3,[RD74-TEDA__nextmark-concat],0.466877,0.478515,0.888712,0.482972,0.506889,0.877384
36,P19,3,[RD74-TEDA__pp_single_mark-concat],0.456140,0.469248,0.889466,0.503356,0.502499,0.889877
26,P19,3,[RD74-TL-TEDA__pp_single_mark-concat],0.463576,0.455533,0.885705,0.484277,0.481607,0.876582


Best-Test-pred_label/f1-binary  \
dataset run_name                                                                
P12     [RD74-DA__base-concat]                                       0.499688   
        [RD74-TEDA__nextmark-concat]                                 0.502558   
        [RD74-TEDA__none-concat]                                     0.499155   
        [RD74-TEDA__pp_ml-concat]                                    0.490677   
        [RD74-TEDA__pp_single_mark-concat]                           0.520503   
P19     [RD74-DA__base-concat]                                       0.478474   
        [RD74-TEDA__nextmark-concat]                                 0.496996   
        [RD74-TEDA__none-concat]                                     0.493503   
        [RD74-TEDA__pp_ml-concat]                                    0.499848   
        [RD74-TEDA__pp_single_mark-concat]                           0.499133   
        [RD74-TL-TEDA__pp_single_mark-concat]                        0.499207   

                                               Best-Test-pred_label/AUPRC  \
dataset run_name                                                            
P12     [RD74-DA__base-concat]                                   0.534171   
        [RD74-TEDA__nextmark-concat]                             0.522394   
        [RD74-TEDA__none-concat]                                 0.521233   
        [RD74-TEDA__pp_ml-concat]                                0.509345   
        [RD74-TEDA__pp_single_mark-concat]                       0.537549   
P19     [RD74-DA__base-concat]                                   0.488444   
        [RD74-TEDA__nextmark-concat]                             0.504753   
        [RD74-TEDA__none-concat]                                 0.492714   
        [RD74-TEDA__pp_ml-concat]                                0.501416   
        [RD74-TEDA__pp_single_mark-concat]                       0.498422   
        [RD74-TL-TEDA__pp_single_mark-concat]                    0.502126   

                                               Best-Test-pred_label/AUROC  \
dataset run_name                                                            
P12     [RD74-DA__base-concat]                                   0.854323   
        [RD74-TEDA__nextmark-concat]                             0.853522   
        [RD74-TEDA__none-concat]                                 0.853264   
        [RD74-TEDA__pp_ml-concat]                                0.852539   
        [RD74-TEDA__pp_single_mark-concat]                       0.861817   
P19     [RD74-DA__base-concat]                                   0.870971   
        [RD74-TEDA__nextmark-concat]                             0.881343   
        [RD74-TEDA__none-concat]                                 0.874688   
        [RD74-TEDA__pp_ml-concat]                                0.875768   
        [RD74-TEDA__pp_single_mark-concat]                       0.881304   
        [RD74-TL-TEDA__pp_single_mark-concat]                    0.877810   

                                               Best-Valid-pred_label/f1-binary  \
dataset run_name                                                                 
P12     [RD74-DA__base-concat]                                        0.517781   
        [RD74-TEDA__nextmark-concat]                                  0.528339   
        [RD74-TEDA__none-concat]                                      0.524506   
        [RD74-TEDA__pp_ml-concat]                                     0.510967   
        [RD74-TEDA__pp_single_mark-concat]                            0.533678   
P19     [RD74-DA__base-concat]                                        0.471288   
        [RD74-TEDA__nextmark-concat]                                  0.472221   
        [RD74-TEDA__none-concat]                                      0.465908   
        [RD74-TEDA__pp_ml-concat]                                     0.478713   
        [RD74-TEDA__pp_single_mark-concat]                            0.491390   
     

NameError: name 'term' is not defined

# CIF

In [ ]:
df_filt = dl_runs(runs, selected_tag='CIF')

df_filt.iloc[0].config.keys()

In [ ]:
i_groupby_row =['dataset','setting',]
i_groupby_col = ['state']
i_agg = ['test_center','split']
# i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)#.dropna(axis=1,how='all')
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df





a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),3))  + ' (' + str(round(x.std(),3))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")